In [18]:
%%file feature_extractor.py
# -*- coding: utf-8 -*-

from __future__ import unicode_literals

from sklearn.feature_extraction.text import TfidfVectorizer
import string
import unicodedata
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.decomposition import LatentDirichletAllocation

# def document_preprocessor(doc):
#     # TODO: is there a way to avoid these encode/decode calls?
#     try:
#         doc = unicode(doc, 'utf-8')
#     except NameError:  # unicode is a default on python 3
#         pass
#     doc = unicodedata.normalize('NFD', doc)
#     doc = doc.encode('ascii', 'ignore')
#     doc = doc.decode("utf-8")
#     return str(doc)

def token_processor(tokens):
    stemmer = SnowballStemmer('english')
    for token in tokens:
        yield stemmer.stem(token)

class FeatureExtractor(TfidfVectorizer):
    """Convert a collection of raw docs to a matrix of TF-IDF features. """

    def __init__(self):
        # see ``TfidfVectorizer`` documentation for other feature
        # extraction parameters.
        super(FeatureExtractor, self).__init__(
                analyzer='word',ngram_range = (1,2),
                stop_words='english', decode_error='replace',
                strip_accents='unicode',
                max_df = 1.0, min_df = 0.0001)
        self.lda = LatentDirichletAllocation(n_components=7, max_iter=20,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=None)

    def fit(self, X_df, y=None):
        """Learn a vocabulary dictionary of all tokens in the raw documents.

        Parameters
        ----------
        X_df : pandas.DataFrame
            a DataFrame, where the text data is stored in the ``statement``
            column.
        """
        X_tf = super(FeatureExtractor, self).fit_transform(X_df.statement)
        self.lda.fit(X_tf)
        return self

    def fit_transform(self, X_df, y=None):
        
        super(FeatureExtractor, self).fit(X_df)
        
        X_tf = self.transform(X_df)
        return self.lda.transform(X_tf)

    def transform(self, X_df):
        X = super(FeatureExtractor, self).transform(X_df.statement)
        return self.lda.transform(X)
    
    def build_tokenizer(self):
        """
        Internal function, needed to plug-in the token processor, cf.
        http://scikit-learn.org/stable/modules/feature_extraction.html#customizing-the-vectorizer-classes
        """
        tokenize = super(FeatureExtractor, self).build_tokenizer()
        return lambda doc: list(token_processor(tokenize(doc)))




Writing feature_extractor.py


In [19]:
!ramp_test_submission --submission wyz

Traceback (most recent call last):
  File "C:\Users\wuyunzhi\Anaconda2\envs\p3\Scripts\ramp_test_submission-script.py", line 11, in <module>
    load_entry_point('ramp-workflow==0.2.0+10.g0b604a6', 'console_scripts', 'ramp_test_submission')()
  File "C:\Users\wuyunzhi\Anaconda2\envs\p3\lib\site-packages\ramp_workflow-0.2.0+10.g0b604a6-py3.5.egg\rampwf\utils\command_line.py", line 85, in ramp_test_submission
    retrain=retrain)
  File "C:\Users\wuyunzhi\Anaconda2\envs\p3\lib\site-packages\ramp_workflow-0.2.0+10.g0b604a6-py3.5.egg\rampwf\utils\testing.py", line 534, in assert_submission
    problem = assert_read_problem(ramp_kit_dir)
  File "C:\Users\wuyunzhi\Anaconda2\envs\p3\lib\site-packages\ramp_workflow-0.2.0+10.g0b604a6-py3.5.egg\rampwf\utils\testing.py", line 77, in assert_read_problem
    problem = imp.load_source(module_name, join(ramp_kit_dir, 'problem.py'))
  File "C:\Users\wuyunzhi\Anaconda2\envs\p3\lib\imp.py", line 172, in load_source
    module = _load(spec)
  File "<froz

In [17]:
cd ..

C:\Users\wuyunzhi\Documents\course\datacamp


In [1]:
import os
import pandas as pd
import numpy as np
import rampwf as rw
from datetime import timedelta

problem_title = 'personality prediction'
_target_column_name = 'type'
_prediction_label_names = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
dict = {'INFJ':0, 'ENTP':1, 'INTP':2, 'INTJ':3, 'ENTJ':4, 'ENFJ':5, 'INFP':6, 'ENFP':7, 'ISFP':8, 'ISTP':9,
 'ISFJ':10, 'ISTJ':11, 'ESTP':12, 'ESFP':13, 'ESTJ':14, 'ESFJ':15}
